In [1]:
# forcing the notebook to reload the modules
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../../')

In [48]:
import gradio as gr
from src.data import datarenderer as dr
from src.data import datafilter as df

import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.data import Dataset
import matplotlib.pyplot as plt
from src.data import fontdb_handler as fh
import datetime
from contextlib import redirect_stdout
from src.model import helperfunctions as hf
import PIL
from zipfile import ZipFile

charset_in = "AaOoUu8Bj"
charset_out = "ÄäÖöÜüß"
box_size = 64

class AutoEncoder(tf.keras.Model):

  def __init__(self, latent_dim, *args, **kwargs):
    assert isinstance(latent_dim, int)
    super().__init__(*args, **kwargs)
    self.latent_dim = latent_dim

    self.inverter_layer = lambda x: 1 - x

    # Encoder
    self.encoder_conv2d_1 = layers.Conv2D(32, (3, 3), activation="relu", padding="same")
    self.encoder_maxpool_1 = layers.MaxPooling2D((2, 2), padding="same")
    self.encoder_conv2d_2 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")
    self.encoder_maxpool_2 = layers.MaxPooling2D((2, 2), padding="same")
    self.encoder_dropout_1 = layers.Dropout(0.1)
    self.encoder_conv2d_3 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")
    self.encoder_maxpool_3 = layers.MaxPooling2D((2, 2), padding="same")
    self.encoder_flatten = layers.Flatten()
    self.encoder_dropout_2 = layers.Dropout(0.1)
    self.encoder_fc1 = layers.Dense(512, activation="relu")
    self.encoder_dropout_3 = layers.Dropout(0.1)
    self.encoder_fc2 = layers.Dense(latent_dim, activation="relu")

    # Decoder
    self.decoder_dropout_1 = layers.Dropout(0.1)
    self.decoder_fc1 = layers.Dense(512, activation="relu")
    self.decoder_reshape = layers.Reshape((4, 4, 32))
    self.decoder_upsample_1 = layers.UpSampling2D((2, 2))
    self.decoder_conv2d_1 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")
    self.decoder_dropout_2 = layers.Dropout(0.1)
    self.decoder_upsample_2 = layers.UpSampling2D((2, 2))
    self.decoder_conv2d_2 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")
    self.decoder_dropout_3 = layers.Dropout(0.1)
    self.decoder_upsample_3 = layers.UpSampling2D((2, 2))
    self.decoder_conv2d_3 = layers.Conv2D(32, (3, 3), activation="relu", padding="same")
    self.decoder_out = layers.Conv2DTranspose(len(charset_out), (3, 3), strides=(2, 2), padding="same")

    self._build_graph()

  def _build_graph(self): # Just here because we want to see the output shapes in the summary.
    input_shape = (box_size, box_size, len(charset_in))
    self.build( (None,) + input_shape )
    inputs = tf.keras.Input(shape=input_shape)
    _ = self.call(inputs)

  def call(self, x):
    z = self.encode(x)
    y = self.decode(z)
    return y

  def encode(self, x):
    encoded = self.inverter_layer(x)
    encoded = self.encoder_conv2d_1(encoded)
    encoded = self.encoder_maxpool_1(encoded)
    encoded = self.encoder_conv2d_2(encoded)
    encoded = self.encoder_maxpool_2(encoded)
    #encoded = self.encoder_dropout_1(encoded)
    encoded = self.encoder_conv2d_3(encoded)
    encoded = self.encoder_maxpool_3(encoded)
    encoded = self.encoder_flatten(encoded)
    #encoded = self.encoder_dropout_2(encoded)
    encoded = self.encoder_fc1(encoded)
    #encoded = self.encoder_dropout_3(encoded)
    encoded = self.encoder_fc2(encoded)
    return encoded

  def decode(self, decoded):
    #decoded = self.decoder_dropout_1(decoded)
    decoded = self.decoder_fc1(decoded)
    decoded = self.decoder_reshape(decoded)
    decoded = self.decoder_upsample_1(decoded)
    decoded = self.decoder_conv2d_1(decoded)
    #decoded = self.decoder_dropout_2(decoded)
    decoded = self.decoder_upsample_2(decoded)
    decoded = self.decoder_conv2d_2(decoded)
    #decoded = self.decoder_dropout_3(decoded)
    decoded = self.decoder_upsample_3(decoded)
    decoded = self.decoder_conv2d_3(decoded)
    decoded = self.decoder_out(decoded)
    decoded = self.inverter_layer(decoded)
    return decoded


def generate_umlaute(file):
    array_chars_in = dr.render_fonts([file], chars=charset_in, size=box_size, normalize=True)
    array_chars_out = model.predict(array_chars_in)
    temp = 0.05
    prediction = 1 / (1 + np.exp(-(array_chars_out[0,:,:,:] - 0.5)/temp))
    return prediction

def plot_char_table(file, umlaut_array=None):
    """
    Plots the character table for the given font file. Like
      0 1 2 3 4 5 6 7 8 9

      A B C D E F G H I J
      K L M N O P Q R S T
      U V W X Y Z 
      
      a b c d e f g h i j
      k l m n o p q r s t
      u v w x y z

      Ä ä Ö ö Ü ü ß

    Args:
        file (str): path to the font file
    """
    chars_numbers = "0123456789"
    chars_upper = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    chars_lower = "abcdefghijklmnopqrstuvwxyz"
    chars_umlaut = "ÄäÖöÜüß"

    fig, axs = plt.subplots(11, 10, figsize=(7.5,15))
    if umlaut_array is None:
        suptitle = "Table of characters for font " + os.path.basename(file)
    else:
        suptitle = "Complete table of characters for font " + os.path.basename(file)
    fig.suptitle(suptitle)

    for i in range(10):
        axs[0, i].imshow(
            dr.render_font(file, size=box_size, chars=chars_numbers[i], normalize=True)[:,:,0],
            cmap="gray"
        )
        axs[0, i].axis('off')
        # Titles in light gray
        axs[0, i].set_title(chars_numbers[i], color="lightgray")
    for i in range(10):
        # Empty plot
        axs[1, i].axis('off')
    for i in range(30):
        if i < len(chars_upper):
            axs[2 + i // 10, i % 10].imshow(
                dr.render_font(file, size=box_size, chars=chars_upper[i], normalize=True)[:,:,0],
                cmap="gray"
            )
            axs[2 + i // 10, i % 10].axis('off')
            axs[2 + i // 10, i % 10].set_title(chars_upper[i], color="lightgray")
        else:
            # Empty plot
            axs[2 + i // 10, i % 10].axis('off')
    for i in range(10):
        # Empty plot
        axs[5, i].axis('off')
    for i in range(30):
        if i < len(chars_lower):
            axs[6 + i // 10, i % 10].imshow(
                dr.render_font(file, size=box_size, chars=chars_lower[i], normalize=True)[:,:,0],
                cmap="gray"
            )
            axs[6 + i // 10, i % 10].axis('off')
            axs[6 + i // 10, i % 10].set_title(chars_lower[i], color="lightgray")
        else:
            # Empty plot
            axs[6 + i // 10, i % 10].axis('off')
    for i in range(10):
        # Empty plot
        axs[9, i].axis('off')
    for i in range(10):
        if i < len(chars_umlaut):
            if umlaut_array is None:
                axs[10, i].imshow(
                    dr.render_font(file, size=box_size, chars=chars_umlaut[i], normalize=True)[:,:,0],
                    cmap="gray"
                )
            else:
                axs[10, i].imshow(
                    umlaut_array[:,:,i],
                    cmap="gray"
                )
            axs[10, i].axis('off')
            axs[10, i].set_title(chars_umlaut[i], color="lightgray")
        else:
            # Empty plot
            axs[10, i].axis('off')
          
    fig.canvas.draw()
    return PIL.Image.frombytes('RGB', fig.canvas.get_width_height(),fig.canvas.tostring_rgb())

def get_font_file(file):
    """
    Loads the font file and displays the character table and the text analysis.
    """
    # Displaying the character table
    image = plot_char_table(file)
    # Displaying the text analysis
    text_analysis = analyse_font_file(file)
    return image, text_analysis

def plot_repaired_table(file, umlaut_array):
    image = plot_char_table(file, umlaut_array)
    return image

def analyse_font_file(file):
    return df.analyse_font_file(file)

def repair_table(file):
    umlaut_array = generate_umlaute(file)
    image = plot_repaired_table(file, umlaut_array)
    # Saving the arrays as zip file. Shape: (64, 64, 8)
    file_name = os.path.basename(file)
    file_name = f"generated_characters_for_{file_name}.zip"
    with ZipFile(file_name, "w") as zip_file:
        for i, char in enumerate(list(charset_out)):
            PIL.Image.fromarray(np.uint8(umlaut_array[:,:,i] * 255)).filter(PIL.ImageFilter.GaussianBlur(radius=0.5)).save(f"{char}.png")
            zip_file.write(f"{char}.png")
            os.remove(f"{char}.png")
    return image, file_name

with gr.Blocks() as demo:
    font_file = gr.File(file_types=[".ttf", ".otf", ".woff", ".woff2"])
    load_file_button = gr.Button("Load file")
    glyph_table_raw = gr.Image()
    text_analysis = gr.Text()
    repair_button = gr.Button("Repair Font File")
    glyph_table_repaired = gr.Image()
    download_file = gr.File(label="Download", interactive=False)

    load_file_button.click(get_font_file, inputs=font_file, outputs=[glyph_table_raw, text_analysis])
    repair_button.click(repair_table, inputs=font_file, outputs=[glyph_table_repaired, download_file])
   
    
if __name__ == "__main__":
    # loading the keras model
    model = tf.keras.models.load_model("../../../models/20231125_174320_val_loss_0.0174_train_loss_0.0138_model.keras",
                                       custom_objects={"AutoEncoder": AutoEncoder})
    demo.launch(show_api=False)   

Running on local URL:  http://127.0.0.1:7895

To create a public link, set `share=True` in `launch()`.


/tmp/ipykernel_29048/2870543135.py:200: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  return PIL.Image.frombytes('RGB', fig.canvas.get_width_height(),fig.canvas.tostring_rgb())


1/1 [==============================] - 0s 153ms/step


/home/johannes/AI_Font_Generator/src/app/gradio/../../../src/data/datarenderer.py:70: RuntimeWarning: overflow encountered in cast
  arrays = np.empty((len(font_file_paths), size, size, len(chars))).astype(dtype)
